In [ ]:
import pandas as pd
from openpyxl.utils import column_index_from_string

df_resin = pd.read_excel("GWPfinal_resinresult.xlsx")
df_result = pd.read_excel("RESULT_undeal5.xlsx")

column_letters = ['AU','AW','BG','BL','BM','BN','BO','BP','BQ','DD','DE','DF','DH','DI','DJ','DK','DL','DM','DN']

column_indices = [column_index_from_string(col_letter) - 1 for col_letter in column_letters]

df_result_subset = df_result.iloc[:, [df_result.columns.get_loc('NUM')] + column_indices]
df_resin['NUM'] = df_resin['NUM'].astype(float).astype(str).str.strip()
df_result_subset['NUM'] = df_result_subset['NUM'].astype(str).str.strip()



df_merged = pd.merge(df_resin, df_result_subset, on='NUM', how='left')

df_merged.to_excel("mergedata.xlsx", index=False)
print(df_resin['NUM'].head(10))
print(df_result['NUM'].head(10))
print(set(df_resin['NUM']) & set(df_result['NUM']))  



In [ ]:
import pandas as pd

df = pd.read_excel("mergedata.xlsx")
for col in df.columns:
    unique_count = df[col].nunique(dropna=True)  
    print(f"{col}: {unique_count}")


In [ ]:
import pandas as pd
import re

df = pd.read_excel("mergedata.xlsx")
target_columns = df.columns[7:12]


def remove_parentheses(text):
    if isinstance(text, str):
        return re.sub(r'\s*\(.*?\)', '', text)
    return text
for col in target_columns:
    df[col] = df[col].apply(remove_parentheses)

df.to_excel("mergedata_cleaned.xlsx", index=False)


In [ ]:
import pandas as pd


df = pd.read_excel("mergedata_cleaned.xlsx")
categorical_cols = ['Resin Type', 'Location1', 'LCIA_Method1']
df = pd.get_dummies(df, columns=categorical_cols)
lci_cols = ['LCI_source1', 'LCI_source2', 'LCI_source3', 'LCI_source4', 'LCI_source5']

all_sources = set()
for col in lci_cols:
    all_sources.update(df[col].dropna().astype(str).str.strip().unique())


invalid_values = {'N', 'n', 'none', 'nan', 'NaN'}
all_sources = {s for s in all_sources if s not in invalid_values}
for source in all_sources:
    colname = f"source_{source.lower().replace(' ', '_')}"
    df[colname] = df[lci_cols].apply(lambda row: int(source in row.values), axis=1)


df.drop(columns=lci_cols, inplace=True)


df.to_excel("mergedata_onehot_full.xlsx", index=False)


